In [1]:
from __future__ import print_function

import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from boruta import BorutaPy

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [7]:
df1=pd.DataFrame(np.random.randn(4,4),index=list('ABCD'),columns=list('ABCD'))
df1['E'] = 0
df1

,A,B,C,D,E
A,-2.134439,0.798028,3.147507,-0.909644,0
B,1.001436,-0.268789,1.063995,-0.437113,0
C,-0.875451,0.776709,0.255124,0.244353,0
D,-0.399723,0.142154,-0.909425,0.092408,0


# read data

In [3]:
final_path = './data/'
# train_merchant
train = pd.read_csv(final_path+'deepctr_feature_train.csv')
#test = pd.read_csv(final_path+'deepctr_feature_test.csv')
#target = pd.read_csv(final_path+'target.csv', index_col=0, names=['target'])

/home/njh/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
train = train.fillna(-1, )
#test = test.fillna(-1, )

In [5]:
features2 = train.columns
features2 = features2.drop('HasDetections')
features2 = features2.drop('MachineIdentifier')
features=features2.tolist()

# start selcetion

In [1]:
rfc = RandomForestRegressor(n_estimators=450, n_jobs= -1, max_depth=9)
boruta_selector = BorutaPy(rfc, n_estimators='auto', verbose=2)
start_time = timer(None)
boruta_selector.fit(train[features].values, target['HasDetections'].values)
timer(start_time)

NameError: name 'RandomForestRegressor' is not defined

# Post-processing

In [6]:
print('\n Initial features: ', train.drop(
    [x for x in train.columns if x in FEATS_EXCLUDED], axis=1).columns.tolist())

# number of selected features
print('\n Number of selected features:')
print(boruta_selector.n_features_)

feature_df = pd.DataFrame(train.drop(
    [x for x in train.columns if x in FEATS_EXCLUDED], axis=1).columns.tolist(), columns=['features'])
feature_df['rank'] = boruta_selector.ranking_
feature_df = feature_df.sort_values(
    'rank', ascending=True).reset_index(drop=True)


 Initial features:  ['feature_1', 'feature_2', 'feature_3', 'quarter', 'elapsed_time', 'days_feature1', 'days_feature2', 'days_feature3', 'days_feature1_ratio', 'days_feature2_ratio', 'days_feature3_ratio', 'feature_sum', 'feature_mean', 'feature_max', 'feature_min', 'feature_var', 'hist_subsector_id_nunique', 'hist_merchant_id_nunique', 'hist_merchant_category_id_nunique', 'hist_merchant_group_id_nunique', 'hist_month_nunique', 'hist_month_mean', 'hist_month_min', 'hist_month_max', 'hist_hour_nunique', 'hist_hour_mean', 'hist_hour_min', 'hist_hour_max', 'hist_weekofyear_nunique', 'hist_weekofyear_mean', 'hist_weekofyear_min', 'hist_weekofyear_max', 'hist_weekday_mean', 'hist_day_nunique', 'hist_day_mean', 'hist_day_min', 'hist_purchase_amount_sum', 'hist_purchase_amount_max', 'hist_purchase_amount_min', 'hist_purchase_amount_mean', 'hist_purchase_amount_var', 'hist_purchase_amount_skew', 'hist_installments_sum', 'hist_installments_max', 'hist_installments_mean', 'hist_installments_va

## find category-feature

In [7]:
need_count_feature = train.columns #[x for x in train.columns if x not in FEATS_EXCLUDED]

In [8]:
feature_nunique_df = pd.DataFrame()
feature_nunique_df['features'] = need_count_feature
feature_nunique_df['nunique'] = train.nunique().values

In [9]:
# feature_nunique_df.set_index('features',inplace=True)
try:
    tmp = feature_nunique_df.drop([x for x in train.columns if x in FEATS_EXCLUDED]).reset_index()
    tmp.shape
except:
    print('*************************************************')


*************************************************


In [10]:
feature_df = pd.merge(feature_df,feature_nunique_df,on='features',how='inner')

In [11]:
feature_df

,features,rank,nunique
0,hist_authorized_flag_mean,1,1630
1,category_1_mean,1,7090
2,new_duration_max,1,9970
3,hist_purchase_date_average,1,7289
4,hist_month_diff_mean,1,733
5,hist_weekofyear_nunique,1,52
6,new_month_lag_mean,1,598
7,new_purchase_amount_var,1,14434
8,hist_duration_min,1,2156
9,new_purchase_amount_max,1,8047


## output

In [12]:
print('\n Top %d features:' % boruta_selector.n_features_)
print(feature_df.head(boruta_selector.n_features_))
feature_df.to_csv('boruta-feature-ranking.csv', index=False)

# check ranking of features
print('\n Feature ranking:')
print(boruta_selector.ranking_)

# check selected features
# print ('\n Selected features:')
# print (boruta_selector.support_)

# check weak features
# print ('\n Support for weak features:')
#print (boruta_selector.support_weak_)


 Top 18 features:
                      features  rank  nunique
0    hist_authorized_flag_mean     1     1630
1              category_1_mean     1     7090
2             new_duration_max     1     9970
3   hist_purchase_date_average     1     7289
4         hist_month_diff_mean     1      733
5      hist_weekofyear_nunique     1       52
6           new_month_lag_mean     1      598
7      new_purchase_amount_var     1    14434
8            hist_duration_min     1     2156
9      new_purchase_amount_max     1     8047
10          hist_duration_mean     1     8304
11    hist_purchase_amount_sum     1    13335
12  new_amount_month_ratio_max     1     9442
13        hist_category_1_mean     1     6169
14       hist_installments_sum     1     1623
15         hist_month_diff_max     1       14
16          hist_month_nunique     1       11
17               new_price_var     1   146323

 Feature ranking:
[290 304 309 279 165  15  36 115  33  58 242 232 245 273 280 194  27  27
  16 120   1 12

In [13]:
selected = train.drop([x for x in train.columns if x in FEATS_EXCLUDED], axis=1).columns[boruta_selector.support_]
train = train[selected]
#train['id'] = tr_ids
#train['target'] = y
#train = train.set_index('id')
train.to_csv('./feature_selection/train_boruta_filtered.csv', index=False)
test = test[selected]
#test['id'] = te_ids
#test = test.set_index('id')
test.to_csv('./feature_selection/test_boruta_filtered.csv', index=False)

In [14]:
train

,hist_month_nunique,hist_weekofyear_nunique,hist_purchase_amount_sum,hist_installments_sum,hist_month_diff_max,hist_month_diff_mean,hist_authorized_flag_mean,hist_category_1_mean,hist_duration_mean,hist_duration_min,hist_purchase_date_average,new_purchase_amount_max,new_purchase_amount_var,new_month_lag_mean,new_price_var,new_duration_max,new_amount_month_ratio_max,category_1_mean
0,9,35,-167.40000,4.000000,12,11.830,0.9500,0.000000,-7.61300,-8.875,0.9307,-0.296100,1.843000e-02,1.4790,1.844478e+10,-3.258000,-0.024670,0.000000
1,12,50,-215.40000,543.000000,13,12.860,0.9688,0.088560,-7.90600,-9.650,1.1140,-0.701700,2.068000e-04,1.5000,2.052360e-04,-8.420000,-0.055020,0.088560
2,10,22,-29.17000,0.000044,12,11.836,0.9536,0.000000,-8.04000,-8.760,9.5800,-0.700000,-1.000000e+00,2.0000,-1.000000e+00,-7.703000,-0.063660,0.000000
3,6,20,-50.12000,84.000000,12,11.780,1.0000,0.155900,-7.65600,-8.890,2.0000,-0.567000,4.345000e-03,1.7140,2.249713e-01,-6.800000,-0.047240,0.298800
4,4,17,-75.25000,182.000000,12,11.760,0.9624,0.112800,-6.67000,-8.950,0.8120,0.451000,5.010000e-02,1.5560,6.389634e-02,5.410000,0.037570,0.168300
5,7,16,-13.77000,99.000000,19,18.970,0.9697,0.030300,-7.92000,-12.910,6.0300,-0.000090,7.855000e-02,1.7500,8.380836e-02,-0.001713,-0.000005,0.030300
6,12,48,-185.40000,0.000263,12,11.950,0.9346,0.000000,-8.52000,-8.900,1.5930,-0.688500,3.836000e-04,1.2000,3.843994e+08,-8.260000,-0.057370,0.000000
7,4,8,-13.57000,0.000022,14,13.550,0.8640,0.000000,-8.34000,-10.440,3.7730,-0.697300,4.800000e-06,2.0000,5.139376e+06,-9.766000,-0.049800,0.000000
8,5,6,-8.71000,16.000000,12,12.000,0.7334,0.266600,-6.96500,-8.950,7.4650,-0.580600,5.207000e-03,1.0000,5.174441e-03,-6.970000,-0.048400,0.266600
9,12,33,-77.00000,3.000000,12,11.860,0.8760,0.000000,-8.08600,-8.920,3.6720,-0.666000,1.704000e-03,2.0000,1.694244e+09,-7.996000,-0.055500,0.000000


In [15]:
len(selected)

18

# playground

In [16]:
train_inf_feat = []
for var in train.columns:
    if train[var].max()== np.inf or train[var].max()== -np.inf:
        train_inf_feat.append(var)

print(train_inf_feat)

[]


In [17]:
tttt = train.groupby('hist_numerical_1_skew').apply(lambda x:x.shape[0])

KeyError: 'hist_numerical_1_skew'

In [ ]:
def show_inf_distributioin(df,feat_name,k=3):
    tmp = df[feat_name].unique()
    idx1 = np.argpartition(tmp, k)
    idx2 = np.argpartition(tmp, -k)
    plus_inf_num = sum(df[feat_name] == np.inf)
    minus_inf_num = sum(df[feat_name] == -np.inf)
    print_str = [x for x in tmp[idx1[:k]]]
    print_str += [x for x in tmp[idx2[-k:]]]
    print(var,print_str,
          '== -inf num:%d'%minus_inf_num+'(ratio:%.4f)'%(minus_inf_num/df.shape[0])+
          ', +inf num:%d'%plus_inf_num+'(ratio:%.4f)'%(plus_inf_num/df.shape[0]))
    print('-------------------------')

######## 
k = 2
for var in train_inf_feat:
    show_inf_distributioin(train, var, k)

In [20]:
np.sqrt(3.679**2-(19**2)/37086)

3.677676830353887